Names: Bryce Hepner, Caelan Osman, Carter Landon
Project: We are trying to predict when flights will be late based on a dataset found from Kaggle.
The dataset includes flight data from 2016 and 2017

In [4]:
import numpy as np
import pandas as pd
import sklearn